<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 右上 select

In [ ]:
import dash
from dash import Dash, dash_table, html, dcc, Input, Output, State
import pandas as pd

# 建立資料（7欄）
data = pd.DataFrame({
    'ID': ['1', '2', '3'],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': ['25', '30', '35'],
    'Gender': ['F', 'M', 'M'],
    'Country': ['USA', 'Canada', 'UK'],
    'Score': ['88', '92', '79'],
    'Passed': ['Yes', 'Yes', 'No']
})
# 所有欄位 & 可切換欄位
all_columns = data.columns.tolist()
toggle_columns = ['Country', 'Score', 'Passed']
default_columns = [col for col in all_columns if col not in toggle_columns]

# 建立 Dash 應用
app = Dash(__name__)

# 設定 layout

app.layout = html.Div([
    html.Div([
        dcc.Checklist(
            id='select-all',
            options=[{'label': 'Select All', 'value': 'all'}],
            value=[],
            style={'display': 'inline-block', 'margin-right': '20px'}
        ),

        html.Div([
            html.Label("Select columns to display:", style={'margin-right': '8px', 'font-weight': 'bold', 'white-space': 'nowrap'}),
            dcc.Checklist(
                id='column-toggle',
                options=[{'label': col, 'value': col} for col in toggle_columns],
                value=[],
                inline=True,
                style={'display': 'inline-block'}
            )
        ], style={'display': 'flex', 'align-items': 'center'})
    ], style={
        'display': 'flex',
        'justifyContent': 'space-between',
        'alignItems': 'center',
        'margin-bottom': '10px'
    }),

    # 資料表
    dash_table.DataTable(
        id='filterable-table',
        columns=[{'name': col, 'id': col} for col in default_columns],
        data=data.to_dict('records'),
        filter_action='native',
        row_selectable='multi',
        selected_rows=[],
        page_size=5,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )
])

# Select All 控制行選取
@app.callback(
    Output('filterable-table', 'selected_rows'),
    Input('select-all', 'value'),
    State('filterable-table', 'data')
)
def update_select_all(value, rows):
    if 'all' in value:
        return list(range(len(rows)))
    return []

# 欄位切換控制 table 欄位顯示
@app.callback(
    Output('filterable-table', 'columns'),
    Input('column-toggle', 'value')
)
def update_table_columns(selected_optional_cols):
    final_cols = default_columns + selected_optional_cols
    return [{'name': col, 'id': col} for col in final_cols]

# 啟動伺服器
if __name__ == '__main__':
    app.run(debug=True)


## 2. 右上 dropdowm

In [ ]:
import dash
from dash import Dash, dash_table, html, dcc, Input, Output, State
import pandas as pd

# 資料
data = pd.DataFrame({
    'ID': ['1', '2', '3'],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Age': ['25', '30', '35'],
    'Gender': ['F', 'M', 'M'],
    'Country': ['USA', 'Canada', 'UK'],
    'Score': ['88', '92', '79'],
    'Passed': ['Yes', 'Yes', 'No']
})

all_columns = data.columns.tolist()
toggle_columns = ['Country', 'Score', 'Passed']
default_columns = [col for col in all_columns if col not in toggle_columns]

# 建立 App
app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        # 左上 Select All
        html.Div(
            dcc.Checklist(
                id='select-all',
                options=[{'label': 'Select All', 'value': 'all'}],
                value=[],
                style={'display': 'inline-block'}
            ),
            style={'display': 'flex', 'alignItems':'flex-end', 'height': '40px'}  # ✅ 對齊高度
        ),
        # 右上欄位顯示 Dropdown
        dcc.Dropdown(
            id='column-toggle',
            options=[{'label': col, 'value': col} for col in toggle_columns],
            value=[],  # 預設無選擇
            multi=True,
            placeholder="Select columns to display...",
            style={'width': '300px'}
        ),
    ], style={
        'display': 'flex',
        'justifyContent': 'space-between',
        # 'alignItems': 'center',
        'alignItems': 'flex-end',
        'margin-bottom': '10px'
    }),

    dash_table.DataTable(
        id='filterable-table',
        columns=[{'name': col, 'id': col} for col in default_columns],
        data=data.to_dict('records'),
        filter_action='native',
        row_selectable='multi',
        selected_rows=[],
        page_size=5,
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )
])

# 選取列（Select All）
@app.callback(
    Output('filterable-table', 'selected_rows'),
    Input('select-all', 'value'),
    State('filterable-table', 'data')
)
def update_select_all(value, rows):
    if 'all' in value:
        return list(range(len(rows)))
    return []

# 顯示欄位控制
@app.callback(
    Output('filterable-table', 'columns'),
    Input('column-toggle', 'value')
)
def update_table_columns(selected_cols):
    final_cols = default_columns + selected_cols
    return [{'name': col, 'id': col} for col in final_cols]

if __name__ == '__main__':
    app.run(debug=True)


## 3. 圓餅圖 固定大小

In [ ]:
# 1. 在 go.Pie 裡面固定 domain 範圍（確定圓餅圖位置及大小）
fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    domain=dict(x=[0, 1], y=[0, 1])  # x, y 的範圍都定死 0~1
)])

# 2. 固定圖表的 layout 尺寸，指定 width 和 height
fig.update_layout(width=400, height=400)

# 3. 在 Dash 佈局中包裝 dcc.Graph 的外層容器設定固定寬高
html.Div(
    dcc.Graph(figure=fig),
    style={'width': '400px', 'height': '400px'}
)

#### 自適應大小圓餅圖

dcc.Graph 設定 style={'width': '100%', 'height': '100%'}，讓 Plotly 圖表充滿外層容器。

In [ ]:
app = dash.Dash(__name__)

labels = ['A', 'B', 'C', 'D']
values = [random.randint(10, 100) for _ in labels]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])

# 移除固定寬高，讓圖表用百分比寬度自適應
fig.update_layout(
    margin=dict(t=40, b=40, l=40, r=40),
    # 不設定 width, height，讓它隨容器尺寸調整
)

app.layout = html.Div([
    html.H3("響應式圓餅圖"),
    html.Div(
        dcc.Graph(
            figure=fig,
            style={'width': '100%', 'height': '100%'}
        ),
        style={'width': '50vw', 'height': '50vw', 'maxWidth': '600px', 'maxHeight': '600px', 'margin': 'auto'}
        # 50vw 寬高，並限制最大尺寸，居中顯示
    )
])

if __name__ == '__main__':
    app.run(debug=True)
